<a href="https://colab.research.google.com/github/dianehkim/StockMarketForecasting/blob/main/Project_Code_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.40.0.2-py2.py3-none-any.whl size=177693439 sha256=7ee92a13ed68667272ebd7aef1901f17862b515c70fc1792b213ca699715f881
  Stored in directory: /root/.cache/pip/wheels/b2/79/e3/842b81607eb31946ee24898cc9961b101e6486f988a5103967
Successfully built h2o


In [ ]:
import pandas as pd
import numpy as np
import h2o
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
data = pd.read_csv("AMZN.csv")
data.dropna()
data.isna().sum()
len(data)

6438

In [ ]:
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplkxmr52d
  JVM stdout: /tmp/tmplkxmr52d/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmplkxmr52d/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_unknownUser_njwtzs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
data.head()
data = data.dropna()
data = data.drop(columns = ["Date"])

In [ ]:
# Remove spaces from column names
data.columns = [col.replace(" ", "") for col in data.columns]


# Display the updated column names
print(data.columns)

Index(['Low', 'Open', 'Volume', 'High', 'Close', 'AdjustedClose'], dtype='object')


In [ ]:
# Split dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Convert pandas dataframes to H2O dataframes
h2o_train_data = h2o.H2OFrame(train_data)
h2o_test_data = h2o.H2OFrame(test_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
y = "AdjustedClose"
x = train_data.drop(columns=[y]).columns

In [ ]:
# Perform feature selection using RFE
rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=5, step=1)
rfe_selector.fit(train_data[x], train_data[y])
selected_features = x[rfe_selector.support_]

In [ ]:
# Update the feature list for H2O
x = selected_features.tolist()

In [ ]:
# Run H2O AutoML
aml = H2OAutoML(max_models=30, seed=1)
aml.train(x=x, y=y, training_frame=h2o_train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_5_AutoML_2_20230402_233114


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    142                142                         102389                 6            6            6             17            64            52.7183

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.002603209775042282
RMSE: 0.05102165986169288
MAE: 0.03401164843185434
RMSLE: 0.00583053704896858
Mean Residual Deviance: 0.002603209775042282

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.0356551653725957
RMSE: 0.18882575399715926
MAE: 0.0821339957724354
RMSLE: 0.007091867817021132
Mean Residual Deviance: 0.0356551653725957

Cross-Validation Metrics Summary: 
                        mean        sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     0.0821018   0.00268247   0.0786574     0.08425       0.0833815     0.0798006     0.0844193
mean_residual_deviance  0.0356868   0.00762606   0.0343879     0.0480301     0.0308566     0.0283402     0.036819
mse                     0.0356868   0.00762606   0.0343879     0.0480301     0.0308566     0.0283402     0.036819
r2                      0.999985    2.93455e-06  0.999984      0.99998       0.999987      0.999987      0.999985
residual_deviance       0.0356868   0.00762606   0.0343879     0.0480301     0.0308566     0.0283402     0.036819
rmse                    0.188097    0.0195636    0.18544       0.219158      0.175661      0.168346      0.191883
rmsle                   0.00706146  0.000221933  0.0068249     0.00706249    0.00733       0.00722988    0.00686004

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse         training_mae          training_deviance
---  -------------------  ----------  -----------------  --------------------  --------------------  ---------------------
     2023-04-02 23:33:08  6.633 sec   0.0                48.04715737297139     36.577826201039464    2308.529331623079
     2023-04-02 23:33:08  6.667 sec   5.0                28.37405010658567     21.59970028869446     805.0867194510342
     2023-04-02 23:33:09  6.698 sec   10.0               16.755868239266352    12.755098409389408    280.75912045165484
     2023-04-02 23:33:09  6.731 sec   15.0               9.895371458283051     7.531739097350431     97.91837629740283
     2023-04-02 23:33:09  6.762 sec   20.0               5.844373234965537     4.447511494151018     34.15669850958154
     2023-04-02 23:33:09  6.792 sec   25.0               3.452583170613309     2.626518472204509     11.920330550002248
     2023-04-02 23:33:09  6.822 sec   30.0               2.0418205321282734    1.551553978286322     4.169031085420585
     2023-04-02 23:33:09  6.852 sec   35.0               1.2093033883640734    0.9168994213452617    1.4624146851088289
     2023-04-02 23:33:09  6.883 sec   40.0               0.7189889105527508    0.5421989387476328    0.5169450534978315
     2023-04-02 23:33:09  6.914 sec   45.0               0.43218706387622907   0.3215663293582722    0.1867856581819557
---  ---                  ---         ---                ---                   ---                   ---
     2023-04-02 23:33:09  7.266 sec   100.0              0.06201455402557206   0.039027165065980654  0.003845804910990596
     2023-04-02 23:33:09  7.300 sec   105.0              0.06003379561403174   0.03821911838303492   0.003604056615827337
     2023-04-02 23:33:09  7.340 sec   110.0              0.058862298306311865  0.0376442888480

In [ ]:
best_model = aml.get_best_model()

In [ ]:
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))


model_id                                                      rmse          mse         mae         rmsle    mean_residual_deviance
GBM_5_AutoML_2_20230402_233114                            0.188826    0.0356552   0.082134     0.00709187                 0.0356552
GBM_grid_1_AutoML_2_20230402_233114_model_5               0.196583    0.0386449   0.0438306    0.00215946                 0.0386449
GBM_4_AutoML_2_20230402_233114                            0.208153    0.0433276   0.065856     0.00334803                 0.0433276
StackedEnsemble_AllModels_1_AutoML_2_20230402_233114      0.208515    0.0434785   0.0800222    0.00651304                 0.0434785
GBM_2_AutoML_2_20230402_233114                            0.235253    0.0553439   0.0902232    0.0064655                  0.0553439
GBM_3_AutoML_2_20230402_233114                            0.242847    0.0589744   0.0764651    0.00421022                 0.0589744
StackedEnsemble_BestOfFamily_1_AutoML_2_20230402_233114   0.243011    0.0590

In [ ]:
# Predict on the test set
y_pred = best_model.predict(h2o_test_data).as_data_frame().values.flatten()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Evaluate the model
mse = mean_squared_error(test_data[y], y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.024993849443255573


In [ ]:
# Clean up
h2o.cluster().shutdown()